In [9]:
import sys
sys.path.append('../utils')
import os
import igl, trimesh
import numpy as np
import point_cloud_utils as pcu
import mesh_tools as mt
from fvdb_utils import mesh_to_grid
from PoNQ_grid import PoNQ_grid
from diffusion_tensor import DiffusionTensor
import torch

In [10]:
class MeshSampler:
    def __init__(self, mesh_name = "canyon", wt_path = "../../data/GT_WT", glb_path = "../../data/GT_GLB", clip_uv = False):
        # Load WT mesh
        self.v, self.f = igl.read_triangle_mesh("{}/{}.obj".format(wt_path, mesh_name))
        self.n = igl.per_face_normals(self.v, self.f, np.array([1e-10, 1e-10, 1e-10]))
        
        # Load GLB mesh
        glb_mesh = trimesh.load("{}/{}.glb".format(glb_path, mesh_name), force="mesh", process=False)
        self.glb = glb_mesh
        self.glb_v = glb_mesh.vertices
        self.glb_f = glb_mesh.faces
        self.glb_uv = glb_mesh.visual.uv
        self.to_color = glb_mesh.visual.material.to_color
        self.clip_uv = clip_uv
        print('Material factors: roughness {}, metallic {}'.format(glb_mesh.visual.material.roughnessFactor, glb_mesh.visual.material.metallicFactor))
        self.normalize()
        
    def normalize(self):
        self.v = 2*mt.NDCnormalize(self.v)
        self.glb_v = 2*mt.NDCnormalize(self.glb_v)
    
    def get_closest_watertight_samples(self, grid_points):
        sdf_compute = igl.signed_distance(grid_points, self.v, self.f, return_normals=True)
        normals = sdf_compute[3]
        if (normals != normals).any():
            c_normals = (grid_points-sdf_compute[2])/sdf_compute[0][:, None]
            c_normals /= np.sqrt((c_normals**2).sum(-1, keepdims=True))
            print('warning: {} nan normals'.format((normals != normals).sum()))
            normals[np.isnan(normals.sum(-1))
                    ] = c_normals[np.isnan(normals.sum(-1))]
        return sdf_compute[2], normals
    
    def get_closest_color(self, sampled_points):
        """returns Nx3 colors in [0, 1]"""
        _, glb_fi, glb_bc = pcu.closest_points_on_mesh(sampled_points, self.glb_v, self.glb_f)
        uv_query = pcu.interpolate_barycentric_coords(self.glb_f, glb_fi, glb_bc, self.glb_uv)
        uv_query[np.isnan(uv_query)] = 0
        if (uv_query.min()<0 or uv_query.max()>1) and not self.clip_uv: 
            print(f'WARNING: UV OUT OF BOUNDS {uv_query.min()} {uv_query.max()}')
        if self.clip_uv:
            uv_query = np.clip(uv_query, 0, 1)

        colors = self.to_color(uv_query)
        # return 0*sampled_points+.5
        return colors[:, :3]/255.

    def get_grid_and_samples(self, n=512, device='cuda'):
        grid = mesh_to_grid(self.v, self.f, n, device)
        grid_points = grid.grid_to_world(grid.ijk.double()).jdata.cpu().detach().numpy()
        sampled_points, sampled_normals = self.get_closest_watertight_samples(grid_points)
        sampled_colors = self.get_closest_color(sampled_points)
        return grid, sampled_points, sampled_normals, sampled_colors

In [11]:
def process_GT(name, size=1024, targets=[512, 256, 128, 64, 32, 16], base_save_path="../../data/GT_sparse_tensors/"):
    ms = MeshSampler(name)
    grid, sampled_points, sampled_normals, sampled_colors = ms.get_grid_and_samples(1024, 'cpu')
    grid = grid.to('cuda')
    if name=="house" or name=="small-town":
        print("invert")
        sampled_colors=1-sampled_colors
    sampled_colors = 2*((2*sampled_colors)-1) # scaling important for diffusion model
    large_base_grid = PoNQ_grid(size)
    large_base_grid.from_mesh(grid, sampled_points, sampled_normals, sampled_colors)
    
    # save
    save_path = f"{base_save_path}/{name}"
    for t_size in targets:
        try:
            os.mkdir(save_path)
        except:
            pass
        large_base_grid = large_base_grid.get_pool(size//t_size)
        large_base_grid.compute_local_offset()
        DT = DiffusionTensor.get_tensor_from_data(large_base_grid.grid, large_base_grid.normals, large_base_grid.local_offset, large_base_grid.colors, torch.ones_like(large_base_grid.local_offset[:, :1]))
        size = t_size
        torch.save(DT, f"{save_path}/{t_size}.pt")
        

In [12]:
# os.mkdir('../../data/GT_sparse_tensors')
# names = [e[:-4] for e in os.listdir('../../data/GT_GLB')]
# for name in names:
#     process_GT(name)

In [13]:
DT = torch.load("../../data/GT_sparse_tensors/house/32.pt", weights_only=False)

In [30]:
import fvdb

tens = fvdb.nn.VDBTensor(DT.grid, DT.grid.jagged_like(DT.jdata))


tens.data.jdata += 1

In [32]:
import fvdb

fvdb.cat

AttributeError: module 'fvdb' has no attribute 'cat'

In [9]:
DT.get_global().colored_PC(.1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.8053603…